# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [5]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup


browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()

locator=("id", "ctl05_gv")
try:
    WebDriverWait(browser, 30, 0.5).until(EC.presence_of_element_located(locator))
finally:
    print('')

# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
trs=table.find_all('tr')

bool_so2=False
str_date=''
str_value=''
for tr in trs:
    tds=tr.find_all('td')
    if (len(tds)>0):
        if (len(tds)==5):
            bool_so2=tds[0].text=='SO2'
            if (bool_so2):
                str_date=tds[2].text
                str_value=tds[3].text
        else:
            str_date=tds[0].text
            str_value=tds[1].text
    if (bool_so2):
        if (str_date<='2018/08'):
            print(f"{str_date} / {str_value}")
        


2018/01 / 1.80
2018/02 / 1.90
2018/03 / 2.20
2018/04 / 2.30
2018/05 / 3.10
2018/06 / 2.70
2018/07 / 2.20
2018/08 / 2.40


### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [11]:
items=[]
monitor_items=[]
search_item=['SO2','CO']

bool_is_search_item=False
for tr in trs:
    tds=tr.find_all('td')
    if (len(tds)>0):
        if (len(tds)==5):
            bool_is_search_item=tds[0].text in search_item
            if (bool_is_search_item):
                items.append(tds[0].text)
                monitor_item={}
                monitor_items.append(monitor_item)
                str_date=tds[2].text
                str_value=tds[3].text
        else:
            str_date=tds[0].text
            str_value=tds[1].text
    if (bool_is_search_item):
        if (str_date<='2018/08'):
            monitor_item[str_date]=str_value
d={}
for item, monitor in zip(items, monitor_items):
    d[item]=monitor
print(d)

{'SO2': {'2018/01': '1.80', '2018/02': '1.90', '2018/03': '2.20', '2018/04': '2.30', '2018/05': '3.10', '2018/06': '2.70', '2018/07': '2.20', '2018/08': '2.40'}, 'CO': {'2018/01': '0.34', '2018/02': '0.44', '2018/03': '0.40', '2018/04': '0.38', '2018/05': '0.34', '2018/06': '0.29', '2018/07': '0.21', '2018/08': '0.30'}}
